# Chatbot for website

### Installing the Required Packages

In [1]:
!pip -q install langchain
!pip -q install bitsandbytes accelerate transformers
!pip -q install datasets loralib sentencepiece
!pip -q install pypdf
!pip -q install sentence_transformers
!pip -q install unstructured
!pip -q install tokenizers
!pip -q install faiss-cpu
!pip -q install xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

### Importing the Required Libraries

In [2]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
from langchain import HuggingFacePipeline
from huggingface_hub import notebook_login
import textwrap
import sys
import os
import torch
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

### Passing the URLs and extracting the data

In [3]:
URLs=[
    'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb',
    'https://www.mosaicml.com/blog/mpt-7b',
    'https://stability.ai/blog/stability-ai-launches-the-first-of-its-stablelm-suite-of-language-models',
    'https://lmsys.org/blog/2023-03-30-vicuna/'

]

In [4]:
loaders=UnstructuredURLLoader(urls=URLs)
data=loaders.load()

In [5]:
data

[Document(page_content='Enable JavaScript and cookies to continue', metadata={'source': 'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb'}),
 Document(page_content='Introducing MPT-7B: A New Standard for Open-Source, Commercially Usable LLMs\n\nby \n\nThe MosaicML NLP Team\n\nMay 5, 2023 in Generative AI\n\nShare this post\n\nIntroducing MPT-7B, the first entry in our MosaicML Foundation Series. MPT-7B is a transformer trained from scratch on 1T tokens of text and code. It is open source, available for commercial use, and matches the quality of LLaMA-7B. MPT-7B was trained on the MosaicML platform in 9.5 days with zero human intervention at a cost of ~$200k.\n\nLarge language models (LLMs) are changing the world, but for those outside well-resourced industry labs, it can be extremely difficult to train and deploy these models. This has led to a flurry of activity centered on open-source LLMs, such as the LLaMA series from Meta, the 

In [6]:
len(data)

4

### Splitting the Text into Chunks

In [7]:
text_splitter=CharacterTextSplitter(separator='\n',
                                    chunk_size=1000,
                                    chunk_overlap=200)

In [8]:
text_chunks=text_splitter.split_documents(data)

In [9]:
len(text_chunks)

57

In [10]:
text_chunks[0]

Document(page_content='Enable JavaScript and cookies to continue', metadata={'source': 'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb'})

In [11]:
text_chunks[1]

Document(page_content='Introducing MPT-7B: A New Standard for Open-Source, Commercially Usable LLMs\nby \nThe MosaicML NLP Team\nMay 5, 2023 in Generative AI\nShare this post\nIntroducing MPT-7B, the first entry in our MosaicML Foundation Series. MPT-7B is a transformer trained from scratch on 1T tokens of text and code. It is open source, available for commercial use, and matches the quality of LLaMA-7B. MPT-7B was trained on the MosaicML platform in 9.5 days with zero human intervention at a cost of ~$200k.\nLarge language models (LLMs) are changing the world, but for those outside well-resourced industry labs, it can be extremely difficult to train and deploy these models. This has led to a flurry of activity centered on open-source LLMs, such as the LLaMA series from Meta, the Pythia series from EleutherAI, the StableLM series from StabilityAI, and the OpenLLaMA model from Berkeley AI Research.', metadata={'source': 'https://www.mosaicml.com/blog/mpt-7b'})

In [12]:
text_chunks[2]

Document(page_content='Today, we at MosaicML are releasing a new model series called MPT (MosaicML Pretrained Transformer) to address the limitations of the above models and finally provide a commercially-usable, open-source model that matches (and - in many ways - surpasses) LLaMA-7B. Now you can train, finetune, and deploy your own private MPT models, either starting from one of our checkpoints or training from scratch. For inspiration, we are also releasing three finetuned models in addition to the base MPT-7B: MPT-7B-Instruct, MPT-7B-Chat, and MPT-7B-StoryWriter-65k+, the last of which uses a context length of 65k tokens!\nOur MPT model series is:\nLicensed for commercial use (unlike LLaMA).\nTrained on a large amount of data (1T tokens like LLaMA vs. 300B for Pythia, 300B for OpenLLaMA, and 800B for StableLM).\nPrepared to handle extremely long inputs thanks to ALiBi (we trained on up to 65k inputs and can handle up to 84k vs. 2k-4k for other open source models).', metadata={'sour

In [13]:
text_chunks[3]

Document(page_content="Prepared to handle extremely long inputs thanks to ALiBi (we trained on up to 65k inputs and can handle up to 84k vs. 2k-4k for other open source models).\nOptimized for fast training and inference (via FlashAttention and FasterTransformer)\nEquipped with highly efficient open-source training code.\nWe rigorously evaluated MPT on a range of benchmarks, and MPT met the high quality bar set by LLaMA-7B.\nToday, we are releasing the base MPT model and three other finetuned variants that demonstrate the many ways of building on this base model:\nMPT-7B Base:\nMPT-7B Base is a decoder-style transformer with 6.7B parameters. It was trained on 1T tokens of text and code that was curated by MosaicML's data team. This base model includes FlashAttention for fast training and inference and ALiBi for finetuning and extrapolation to long context lengths.\nLicense: Apache-2.0\nHuggingFace Link: https://huggingface.co/mosaicml/mpt-7b\nMPT-7B-StoryWriter-65k+", metadata={'source

### Downloading Hugging Face Embeddings

In [14]:
#embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
embeddings=HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [15]:
query_result = embeddings.embed_query("Hello world")
len(query_result)


768

### Converting the Text Chunks into Embeddings and Creating a Knowledge Base

In [16]:
vectorstore=FAISS.from_documents(text_chunks, embeddings)

### Creating Llama 2 model wrapper

In [17]:
notebook_login()

In [19]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                          use_auth_token=True,)


model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                              load_in_8bit=True,
                                              #load_in_4bit=True
                                             )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:671: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [20]:
pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [21]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

In [22]:
llm.predict("Please provide a concise summary of the Book Alchemist")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


' by Paulo Coelho.\nIn the book "The Alchemist" by Paulo Coelho, a young shepherd named Santiago travels to Egypt in search of a treasure he believes is buried there. Along the way, he meets various people who teach him about the nature of the universe and the power of belief. He also learns about the importance of following his heart and listening to the universe\'s signs. Through his journey, Santiago discovers that the treasure he seeks is not just a material object, but a spiritual one that represents the fulfillment of his personal legend.\n\nThe book explores themes of spirituality, self-discovery, and the power of belief. It encourages readers to listen to their hearts and follow their dreams, as the universe will provide the necessary signs and guidance. The book also emphasizes the importance of perseverance and determination in achieving one\'s goals.\n\nOverall, "The Alchemist" is a thought-provoking and inspiring tale that encourages readers to embrace their personal legend

### Initialization of the Retrieval QA with Source Chain

In [23]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorstore.as_retriever())

In [24]:
result=chain({"question": "How good is Vicuna?"}, return_only_outputs=True)
result

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1872 > 1024). Running this sequence through the model will result in indexing errors


{'answer': ' Vicuna is good, but the exact level of its quality is unknown and requires further research.\n',
 'sources': ''}

In [25]:
wrapped_text = textwrap.fill(result['answer'], width=500)
wrapped_text

' Vicuna is good, but the exact level of its quality is unknown and requires further research.'

In [26]:
result=chain({"question": "How does Llama 2 outperforms other models"}, return_only_outputs=True)
result

{'answer': ' Llama 2 outperforms other models in the study, including MPT-7B-Instruct, MPT-7B-Chat, MPT-7B-StoryWriter-65k+, LLaMA-7B, Pythia, OpenLLaMA, and StableLM.\n',
 'sources': ''}

In [27]:
wrapped_text = textwrap.fill(result['answer'], width=500)
wrapped_text

' Llama 2 outperforms other models in the study, including MPT-7B-Instruct, MPT-7B-Chat, MPT-7B-StoryWriter-65k+, LLaMA-7B, Pythia, OpenLLaMA, and StableLM.'

In [28]:
result=chain({"question": "What is is stableLM?"}, return_only_outputs=True)
result

{'answer': ' StableLM is a training method for large language models (LLMs) that eliminates the challenges of training LLMs by using a combination of techniques, including automatic handling of dead GPUs and no mid-stream learning rate changes.\n',
 'sources': ''}

In [29]:
wrapped_text = textwrap.fill(result['answer'], width=500)
wrapped_text

' StableLM is a training method for large language models (LLMs) that eliminates the challenges of training LLMs by using a combination of techniques, including automatic handling of dead GPUs and no mid-stream learning rate changes.'

In [30]:
result=chain({"question": "Can you please share some details about MPT-7b Model"}, return_only_outputs=True)
result

{'answer': ' The purpose of MPT-7B is to demonstrate the capabilities of the MosaicML platform for building and deploying NLP models at scale.\n',
 'sources': ''}

In [31]:
wrapped_text = textwrap.fill(result['answer'], width=100)
wrapped_text

' The purpose of MPT-7B is to demonstrate the capabilities of the MosaicML platform for building and\ndeploying NLP models at scale.'

In [32]:
# while True:
#   query=input(f"Prompt: ")
#   if query == 'exit':
#     print('Exiting')
#     sys.exit()
#   if query =='':
#     continue
#   result=chain({'question':query})
#   print(f"Answer: " +result["answer"])
